In [4]:
import requests

### Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса  
https://www.cbr-xml-daily.ru/daily_json.js

### Задание 2

Добавьте в класс `Rate` параметр `diff` (со значениями `True` или `False`), который в случае значения `True` в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, `self.diff` будет принимать значение `True` только при возврате значения курса. При отображении всей информации о валюте он не используется.

### Решение

Для задания 1 функция нахождения валюты с максимальным курсом добавлена в основной класс валют `Rate` и называется `get_max_currency`.
Для задания 2 в конструктор был добавлен параметр `diff` и изменен метод make_format.

In [9]:
class Rate:
    def __init__(self, format_='value', diff = False):
        self.format = format_
        self.diff = diff # Задание 2
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff:
                    return response[currency]['Value'] - response[currency]['Previous'] # Задание 2
                else:
                    return response[currency]['Value']
                       
        return 'Error'
   
    # Задание 1
    def get_max_currency(self):
        response = self.exchange_rates()
        max_value = 0
        max_currency = ''
        for currency in response:
            current_data = response[currency]
            if current_data['Value'] >= max_value:
                max_value = current_data['Value']
                max_currency = current_data['Name']
                
        return f'Валюта - {max_currency}, курс = {max_value}'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [13]:
r = Rate(diff=True)

In [14]:
r.get_max_currency()

'Валюта - СДР (специальные права заимствования), курс = 107.1937'

In [15]:
r.make_format('EUR')

-0.6768000000000001

In [16]:
r.usd()

-0.394999999999996

В первом задании как вариант можно использовать более короткую версию с функцией max:

```
import requests
r = requests.get('<https://www.cbr-xml-daily.ru/daily_json.js>')

data_dict = r.json()['Valute']
max(data_dict.items(), key=lambda x: x[1]['Value'])[1]['Name']

```


### Задание 3

Напишите класс `Designer`, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс `Designer` пишется по аналогии с классом `Developer` из материалов занятия.

In [17]:
# Базовый класс сотрудника
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1 
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [54]:
# Класс для работы с дизайнерами
class Designer(Employee):
    # выполнение условия, что дизайнер по-умолчанию имеет две премии, когда выходит на работу
    def __init__(self, name, seniority = 4):
        super().__init__(name, seniority)
        
    def check_if_it_is_time_for_upgrade(self):
        # для каждого получения международной премии увеличиваем счетчик на 2
        self.seniority += 2
         
        # условие повышения сотрудника из презентации
        # с учетом того, что баллов может быть изначально не кратно 7, ориентируемся на текущий грейд
        # т.е. например, если начнем с 4-х баллов, то на 8-ми баллах грейд будет +1         
        if self.seniority // 7 >= self.grade: 
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [55]:
kate = Designer('Екатерина')

In [56]:
for i in range(20):
    kate.check_if_it_is_time_for_upgrade()

Екатерина 1
Екатерина 2
Екатерина 2
Екатерина 2
Екатерина 3
Екатерина 3
Екатерина 3
Екатерина 3
Екатерина 4
Екатерина 4
Екатерина 4
Екатерина 5
Екатерина 5
Екатерина 5
Екатерина 5
Екатерина 6
Екатерина 6
Екатерина 6
Екатерина 7
Екатерина 7
